In [ ]:
import os
import sys
import pandas as pd
import yaml 
from matplotlib import pyplot as plt
from matplotlib import ticker as mticker
import statsmodels.api as sm
import numpy as np
from itertools import product
import subprocess

with open("../../config.yaml.local", "r") as f:
    LOCAL_CONFIG = yaml.safe_load(f)
#with open("../../config.yaml", "r") as f:
#    CONFIG = yaml.safe_load(f)
sys.path.append("../python")

import globals
import data_tools as dt

LOCAL_PATH = LOCAL_CONFIG["LOCAL_PATH"]
RAW_DATA_PATH = LOCAL_CONFIG["RAW_DATA_PATH"]
DATA_PATH = LOCAL_CONFIG["DATA_PATH"]
R_PATH = LOCAL_CONFIG["R_PATH"]

RUN_R_SCRIPTS = False


In [ ]:
user_stats = dt.get_user_stats_days()
user_stats['week'] = dt.as_week(user_stats['date'])

items = dt.get_items()
items['week'] = dt.as_week(items['created_at'])

In [ ]:
# Initialize user-weekly panel
users = list(items['userId'].unique())
week_start = globals.data_start.to_period('W-SAT').start_time
week_end = globals.data_end.to_period('W-SAT').start_time
weeks = list(pd.date_range(start=week_start, end=week_end, freq='7D'))
df = pd.DataFrame( list(product(users, weeks)), columns=['userId', 'week'])

# Merge on user first item and drop weeks before first item
first_items = items.groupby('userId').agg(
    first_item_week = ('week', 'min')
).reset_index()
df = df.merge(first_items, on='userId', how='left')
df = df.loc[df['week'] >= df['first_item_week']].reset_index(drop=True)
df = df.drop(columns=['first_item_week'])

# User weekly stats
weekly_stats = user_stats.drop(columns=['date'])
weekly_stats = weekly_stats.groupby(['userId', 'week']).sum().reset_index()

# Merge on weekly stats
df = df.merge(weekly_stats, on=['userId', 'week'], how='left')
df = df.fillna(0)

In [ ]:
# define activity as any comment, post, or zap
df['activity'] = (
    (df['comments'] > 0) | 
    (df['posts'] > 0) |
    (df['sats_tipped'] > 0)
)


In [ ]:
user_stats.columns

In [ ]:
user_stats